In [ ]:
import ipywidgets
import pandas as pd

In [ ]:
import pymongo
from bson.objectid import ObjectId

In [ ]:
client = pymongo.MongoClient()

database = 'uwnet'
run_collection_name = 'runs'
metric_collection_name = 'metrics'

In [ ]:
client[database].list_collection_names()

In [ ]:
collection = client[database][run_collection_name]

metrics = client[database][metric_collection_name]d

this is what one record looks like

In [ ]:
runs_with_artifacts = collection\
.find({"$where": "this.artifacts.length > 0"})\
.sort([("start_time", pymongo.DESCENDING)])

run = runs_with_artifacts.next()

# Drop all entries without artifacts
collection.find({"$where": "this.artifacts.length == 0", 'status': {"$ne": "RUNNING"}})
collection.delete_many({"$where": "this.artifacts.length == 0", 'status': {"$ne": "RUNNING"}})

metrics_dict = {m['name']: ObjectId(m['id']) for m in run['info']['metrics']}

loss = metrics.find_one({'_id': metrics_dict['loss']})
loss_series = pd.Series(loss['values'], index=loss['steps'])

loss_series.plot()